# 开机过程
- 1.加载BIOS
    - 系统固件加载BIOS（基本输入输出系统），然后开机自检（POST）,并开始初始化部分硬件
    - BIOS:它是一套程序，写死到主板上的一个内存芯片中，这个内存芯片在没有通电的情况下也能将数据记录下来（ROM,非挥发性）
- 2.读取MBR
    - 系统固件搜索磁盘上的主启动记录（MBR）
    - 由于我们的系统软件大多放置在硬盘中，所以BIOS会指定启动的设备，好让我们可以读取磁盘中的操作系统内核文件，这时候就需要一个引导装载程序来处理内核文件加载（LOAD）的问题，这个引导装载程序就被称为BootLoader
    - 这个BootLoader放在哪里呢？就放在我们系统第一个硬盘上的第一个扇区里，也就是我们说的MBR:master boot record
- 3.启动加载器（grub2）
    - 系统固件从磁盘中读取启动加载器（Boot Loader），然后将系统控制权交给启动加载器，在Redhat7中，我们的BootLoader是通过一个grub程序来实现的
    - 现在是grub 2.0，前一个版本是grub0.97
    - grub2的主要功能
        - 1.提供菜单：选择不同的启动选项
        - 2.加载内核文件：直接指向可启动的程序区段来开始OS
        - 3.转交其他Loader（多系统时）
- 4.显示菜单
    - 启动加载器从我们的磁盘中加载其配置，，然后向用户显示用于启动的可能配置的菜单，这就是我们在开机的时候会看到的让你上下选择什么什么版本啊，什么什么东西的那个界面
    -  可以通过以下方式进行配置
        - /etc/grub.d/
        - /etc/default/grub
        - /boot/grub2/grub.cfg(非手动)
- 5.加载内核
    - 在用户做出选择时（超时）后，启动加载器会从硬盘中加载配置的内核以及initramfs，并将其置于内存中
    - initrams是一个 归档文件，里面包含了启动时所需要的必要硬件的内核模块，初始化脚本等等，在Redhat7中，initrams得到了加强，包含自身可用的整个系统
- 6.系统内部传递信息
    - 这时候我们的启动加载器又将系统控制权交给内核，从而传递启动加载器的内核命令行中指定的任何选项，以及initramfs在内核中的位置
- 7初始化
    - 内核这时候会在initramfs中找到驱动程序的所有硬件，并且初始化它，然后作为PID1从initramfs中执行/sbin/init
    - PID1就是systemd，代表的是系统启动的第一个用户级进程，是所有其他进程的父进程
- 8.执行单元
    - 初始化完成后，我们的systemd就会执行intrd.target目标的所有文件，这些所有的东西都包括在/sysroot上挂载实际的root文件系统中
- 9.系统回归
    - 接下来内核root文件系统会回到之前挂载于/sysroot上的系统root文件系统，然后就是我们systemd的事情了，它会使系统中安装的systemd副本来自行重新执行
- 10.开机界面
    - 就是我们熟悉的输入账号密码的界面了
    

# Root密码破解——基于Redhat7
- 1.开机按e进入编辑模式
- 2.选择Linux16那一行，然后光标会移动到这一行末尾，添加一个参数rd.break
- 3.然后按ctrl+x执行，执行之后会进入一个shell界面
- 4.设置root密码，设置密码一共5步
    - 1.重新挂载根目录为读写模式：mount -o remount,rw /sysroot
    - 2.切换到系统真正的根chroot/sysroot
    - 3.修改密码passwd root
    - 4.SELinux给所有文件夹标签，必须要有.autolabel文件，touch /.autorelabel
    - 5.退出编辑模式（两次），系统会自动重启exit两次